# Installing Dependencies

In [27]:
import sys

# Installing numpy
!{sys.executable} -m pip install numpy

# Installing pandas
!{sys.executable} -m pip install pandas

# Installing matplotlib
!{sys.executable} -m pip install matplotlib

# Installing statsmodels
!{sys.executable} -m pip install statsmodels

#Installing tqdm
!{sys.executable} -m pip install tqdm

#Installing Geopy
!{sys.executable} -m pip install geopy


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Imports

In [28]:
# Import Statistic Libraries
import numpy                    as np
import pandas                   as pd
import matplotlib.pyplot        as plt

# Import Utility Libraries
from typing                     import List, Dict, TypedDict
from tqdm                       import tqdm
from matplotlib.lines           import Line2D
from geopy.geocoders            import Nominatim

# Algorythm

In [29]:
class UserRequest(TypedDict):
    City: str
    MinPrice: float
    MaxPrice: float
    MinSpace: float
    MaxSpace: float
    range: int

In [34]:
users_df: pd.DataFrame = pd.read_csv('../data/users.csv')
requests_df: pd.DataFrame  = pd.read_csv('../data/requests.csv')
listings_df: pd.DataFrame  = pd.read_csv('../data/combined_data.csv',delimiter=";")

listings_df['size'] = listings_df['size'].str.extract('(\d+)').astype(int)
listings_df['price'] = listings_df['price'].str.replace(',', '').astype(int)
listings_df['street'] = listings_df['street'].str.rstrip(',')

listings_df['start_date'] = listings_df['date'].str.extract(r'(?:From (\d+ [a-zA-Z]+ \d+))|(\d+ [a-zA-Z]+ \d+) -').bfill(axis=1).iloc[:, 0]
listings_df['end_date'] = listings_df['date'].str.extract(r' - (\d+ [a-zA-Z]+ \d+)').bfill(axis=1).iloc[:, 0]

listings_df['furnishing'] = listings_df['furnishing'].map({"unfurnished": False, "furnished": True})

listings_df['start_date'] = pd.to_datetime(listings_df['start_date'], errors='coerce', format='%d %b %Y')
listings_df['end_date'] = pd.to_datetime(listings_df['end_date'], errors='coerce', format='%d %b %Y')
listings_df = listings_df.drop('date', axis=1)

geolocator = Nominatim(user_agent="geo_locator")

def get_coordinates(row):
    location = geolocator.geocode(f"{row['street']}, {row['city']}")
    if location:
        return location.latitude, location.longitude
    else:
        return None, None
    
listings_df[['latitude', 'longitude']] = listings_df.apply(get_coordinates, axis=1, result_type='expand')

In [57]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  

    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)

    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance

def match_listings(user_request: pd.DataFrame, listings_df: pd.DataFrame):
    geolocator = Nominatim(user_agent="geo_locator")

    location = geolocator.geocode(user_request['City'])
    user_latitude, user_longitude = location.latitude, location.longitude

    matching_df = listings_df[
        ((listings_df["city"] == user_request["City"])
            | (haversine_distance(user_latitude, user_longitude, listings_df["latitude"], listings_df["longitude"]) < user_request["Range"])) &
        (listings_df['price'] >= user_request['MinPrice'] if not pd.isna(user_request['MinPrice']) else True) &
        (listings_df['price'] <= user_request['MaxPrice'] if not pd.isna(user_request['MaxPrice']) else True) &
        (listings_df['size'] >= user_request['MinSpace'] if not pd.isna(user_request['MinSpace']) else True) &
        (listings_df['size'] <= user_request['MaxSpace']if not pd.isna(user_request['MaxSpace']) else True)
    ].copy()  

    matching_df['distance'] = matching_df.apply(lambda row: haversine_distance(user_latitude, user_longitude, row['latitude'], row['longitude']), axis=1)

    return matching_df

def rank_listings(matching_results):
    matching_results['normalized_price'] = matching_results['price'] / matching_results['size'] # i.e. price per m2 per month
    
    weight_price_per_m2 = 0.4
    weight_distance = 0.6
    
    matching_results['ranking_score'] = (
        (1 / matching_results['normalized_price']) * weight_price_per_m2 +
        (1 / (1 + matching_results['distance'])) * weight_distance
    )
    
    ranked_results = matching_results.sort_values(by='ranking_score', ascending=False)
    ranked_results = ranked_results.reset_index(drop=True)
    
    return ranked_results

In [58]:
user_request: UserRequest = requests_df.iloc[3].to_dict()
print(user_request)
matching_results = match_listings(user_request, listings_df)
ranked_results = rank_listings(matching_results)
print(len(ranked_results))

{'City': 'Rotterdam', 'MinPrice': 900.0, 'MaxPrice': nan, 'MinSpace': 9, 'MaxSpace': nan, 'Range': 40, 'email': 'paarth@paarth.com'}
8


In [59]:
ranked_results[['street','city','size','furnishing','type','price','distance','url']]

,street,city,size,furnishing,type,price,distance,url
0,Graaf Balderikstraat,Rotterdam,28,False,Room,1000,0.409825,https://kamernet.nl/en/for-rent/room-rotterdam...
1,Admiraal de Ruyterweg,Rotterdam,61,True,Apartment,1800,0.577526,https://kamernet.nl/en/for-rent/apartment-rott...
2,Spijkenissestraat,Rotterdam,65,True,Apartment,1100,3.336231,https://kamernet.nl/en/for-rent/apartment-rott...
3,Pleinweg,Rotterdam,85,True,Apartment,1300,3.638020,https://kamernet.nl/en/for-rent/apartment-rott...
4,Meidoornsingel,Rotterdam,20,True,Room,925,3.538104,https://kamernet.nl/en/for-rent/room-rotterdam...
5,Broersvest,Schiedam,40,True,Room,1000,5.274808,https://kamernet.nl/en/for-rent/room-schiedam/...
6,Maaslandse Dam,Maasland,90,True,Apartment,1250,13.806962,https://kamernet.nl/en/for-rent/apartment-maas...
7,Stoofsteeg,Gouda,58,False,Apartment,1325,18.747017,https://kamernet.nl/en/for-rent/apartment-goud...
